In [20]:
# Cargamos los datos de los ficheros y los mostramos

import json


with open('peliculas_humor.json', 'r') as c:
        humor = json.load(c)
print(type(humor))


with open('peliculas_terror.json', 'r') as t:
        terror = json.load(t)
print(type(terror))

<class 'list'>
<class 'list'>


In [21]:
# Hacemos un dataframe con los datos de los dos ficheros

import pandas as pd

todopelis = humor + terror

df = pd.DataFrame(todopelis)
df_total = df.sample(frac=1).reset_index(drop=True)
df_total

,title,genre,sinopsis
0,Cabinet of Curiosities: Graveyard Rats,"[Terror, Comedia]",[Un ladrón de tumbas le echa el ojo a las riqu...
1,Mudbrick,[Terror],[Después de heredar una vieja casa de adobe en...
2,Star Wars Visions: I Am Your Mother,"[Animación, Comedia]",[]
3,Sweeter Than Chocolate,"[Comedia, Romance]",[Un reportero de televisión va a investigar un...
4,Cabinet of Curiosities: Lot 36,[Terror],[Un hombre cree que un trastero lleno de trast...
...,...,...,...
995,American Horror Stories: Lake,"[Terror, Intriga, Drama]",[Un misterioso ahogamiento deja entrever secre...
996,"Ijiranaide, Nagatoro-san 2nd Attack","[Serie de TV, Animación, Comedia, Romance]",[Serie de TV (2023). 12 episodios. Secuela de ...
997,Era ora,"[Drama, Comedia, Fantástico]",[Dante besa accidentalmente a Alice que tiene ...
998,El Tonto,[Comedia],[Un sordomudo (Charlie Day) se convierte en un...


In [22]:
# Creamos un nuevo Dataframe que contiene solo las filas con género Humor o Terror y luego obtengo el recuento de cada género en otro dataframe

df_exploded = df_total.explode('genre') 
genre_counts = df_exploded['genre'].value_counts() 

df_dos_generos = df_exploded[df_exploded['genre'].isin(['Comedia', 'Terror'])] 
genre_counts_dos_generos = df_dos_generos['genre'].value_counts() 

df_dos_generos['sinopsis'] = df_dos_generos['sinopsis'].apply(''.join) 

C:\Users\MSI\AppData\Local\Temp\ipykernel_2080\3995828712.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dos_generos['sinopsis'] = df_dos_generos['sinopsis'].apply(''.join)


In [23]:
# Comenzamos a preparar el modelo dividiendo los datos en train y test
from sklearn.model_selection import train_test_split
sinopsis = df_dos_generos['sinopsis']
genero = df_dos_generos['genre']
sinopsis_train, sinopsis_test, genero_train, genero_test = train_test_split(sinopsis, genero, test_size=0.4, random_state=30)


In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#Creamos el vectorizador sin especificar las stopwords
vectorizador = CountVectorizer()

#Aplicamos el vectorizador a los datos de entrenamiento y prueba
sinopsis_train_vector = vectorizador.fit_transform(sinopsis_train)
sinopsis_test_vector = vectorizador.transform(sinopsis_test)

#Creamos el modelo de clasificación y lo entrenamos usando LogisticRegression
clf = LogisticRegression()
clf.fit(sinopsis_train_vector, genero_train)


preds = clf.predict(sinopsis_test_vector)
print(classification_report(genero_test,preds,target_names=['Humor','Terror']))

              precision    recall  f1-score   support

       Humor       0.71      0.60      0.65       235
      Terror       0.59      0.70      0.64       195

    accuracy                           0.65       430
   macro avg       0.65      0.65      0.65       430
weighted avg       0.66      0.65      0.65       430



In [25]:
# Creamos un pipeline 

from sklearn.pipeline import Pipeline
import pickle as pkl

pipe=Pipeline([('vectorizador',vectorizador),('clasificador',clf)])
pipe.predict([sinopsis_test.iloc[67]])


array(['Comedia'], dtype=object)

In [26]:
# Guardamos el modelo
pkl.dump(pipe, open("clf_genero.pkl", "wb"))